In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.backend import clear_session

import ast

In [ ]:
input_size = 62

In [ ]:
labels =  [int(i.strip()) for i in open("./te_label_sm.dat").readlines()]
len(labels)

750

In [ ]:
# load mnist dataset
data = []

j = 0
i = 0
with open("./te_data_sm.dat") as fd:
  for j in range(750):
    _input = []
    for i in range(62):
      el = fd.readline()
      _input.append(int(el.strip(), 16))
      if _input[i] > 127:
        _input[i] = -(_input[i] - 128)
    data.append(_input)

print(data[0])
print(len(data[0]))
print(len(data))

[-127, -114, -24, -127, -93, 92, 125, 110, 126, -127, -104, 96, 61, -84, -122, 79, -127, -21, 69, -123, -115, 15, 94, -127, -25, 72, -39, 72, 95, -78, -127, -66, 126, 126, 106, -103, -127, 54, 126, 79, -40, 65, -58, -127, 59, -62, -127, -127, -48, 26, -127, 3, -124, -124, -54, 109, 5, 109, 102, 104, 116, 17]
62
750


In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
# compute the number of labels
num_labels = len(np.unique(labels))
num_labels

10

In [ ]:
# convert to one-hot vector
labels = to_categorical(labels, num_labels)
labels[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)

In [ ]:
# network parameters
batch_size = 32

# model is a 3-layer MLP with ReLU and dropout after each layer
model = Sequential(
    [
     Dense(30, input_dim=input_size, activation='relu'),
     Dense(num_labels, activation='softmax')
    ]
)
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                1890      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
Total params: 2,200
Trainable params: 2,200
Non-trainable params: 0
_________________________________________________________________


In [ ]:
clear_session()
# train the network
model.fit(data, labels, epochs=20, batch_size=32)

Epoch 1/20
24/24 [==============================] - 0s 2ms/step - loss: 4.0641 - accuracy: 0.8200
Epoch 2/20
24/24 [==============================] - 0s 2ms/step - loss: 3.3811 - accuracy: 0.8373
Epoch 3/20
24/24 [==============================] - 0s 2ms/step - loss: 2.8553 - accuracy: 0.8507
Epoch 4/20
24/24 [==============================] - 0s 2ms/step - loss: 2.4717 - accuracy: 0.8787
Epoch 5/20
24/24 [==============================] - 0s 2ms/step - loss: 2.1545 - accuracy: 0.8760
Epoch 6/20
24/24 [==============================] - 0s 2ms/step - loss: 1.7905 - accuracy: 0.9053
Epoch 7/20
24/24 [==============================] - 0s 2ms/step - loss: 1.5049 - accuracy: 0.9027
Epoch 8/20
24/24 [==============================] - 0s 2ms/step - loss: 1.2581 - accuracy: 0.9107
Epoch 9/20
24/24 [==============================] - 0s 2ms/step - loss: 1.0716 - accuracy: 0.9253
Epoch 10/20
24/24 [==============================] - 0s 2ms/step - loss: 0.9224 - accuracy: 0.9240
Epoch 11/20
24/24 [

In [ ]:
# validate the model on dataset
_, acc = model.evaluate(data,
                        labels,
                        batch_size=batch_size,
                        verbose=0)
print("Accuracy: %.1f%%" % (100.0 * acc))

Accuracy: 99.1%


In [ ]:
print(len(model.layers[0].get_weights()[0])) # Hidden Layer Weights
print(len(model.layers[0].get_weights()[1])) # Hidden Layer Bias
print(len(model.layers[1].get_weights()[0])) # Output Layer Weights
print(len(model.layers[1].get_weights()[1])) # Output Layer Bias

62
30
30
10


In [ ]:
def convert_to_hex(w_or_b):
  if (len(w_or_b.shape) == 2):
    w_or_b = w_or_b.reshape(w_or_b.shape[1], w_or_b.shape[0])
  hex_arr = []
  for i in range(len(w_or_b)):
    if (len(w_or_b.shape) == 2):
      str = ""
      for j in range(w_or_b.shape[1]):
        if (int(w_or_b[i][j]) < 0):
          str += hex(127-int(w_or_b[i][j])).split('x')[-1].zfill(2)
        else:
          str += hex(int(w_or_b[i][j])).split('x')[-1].zfill(2)
      hex_arr.append(str)
    else:
      if (int(w_or_b[i]) < 0):
        hex_arr.append(hex(127-int(w_or_b[i])).split('x')[-1].zfill(2))
      else:
        hex_arr.append(hex(int(w_or_b[i])).split('x')[-1].zfill(2))
  print(len(hex_arr))
  return hex_arr

In [ ]:
file_names = ["custom_w1_sm.dat", "custom_b1_sm.dat", "custom_w2_sm.dat", "custom_b2_sm.dat"]
k = 0
for i in range(len(model.layers)):
  for j in range(len(model.layers[i].get_weights())):
    print(file_names[k])
    with open(file_names[k], 'w') as fd:
      for str in convert_to_hex(np.array(model.layers[i].get_weights()[j]*255)):
        print(str, file=fd)
    k+=1

custom_w1_sm.dat
30
custom_b1_sm.dat
30
custom_w2_sm.dat
10
custom_b2_sm.dat
10
